In [ ]:
import numpy as np

corr_harmo = np.load('correlation_mat_harmo_whole.npy')

def equal_len(corr_mat): 

    a_dims = np.array([item.shape for item in corr_mat])

    rois_dim = np.max(a_dims[:, 1])
    ts_dim = np.max(a_dims[:, 0])

    x_data = np.zeros((ts_dim, rois_dim, len(corr_mat)))

    for idx, val in enumerate(corr_mat):
        x_data[0:val.shape[0], 0:val.shape[1], idx] = val

    return x_data

def gender_selection(phen_abide, corr_harmo, y_target): 

    '''
    :param phen_abide: phenotypic Pandas Dataframe extracted at the beggining of the notebook

    :param corr_harmo: correlation matrix array with the whole dataset - harmonized

    :param y_target: y_target array with vales [1,0]

    :return: X_data, y_target for male and female respectively
    
    '''

    new_indices = np.arange(0, 172, 1)

    idx_phen_abide = phen_abide.set_index(new_indices)

    males_stat = idx_phen_abide[idx_phen_abide['SEX'] == 1]
    males_index = np.array(males_stat.index)

    females_stat = idx_phen_abide[idx_phen_abide['SEX'] == 2]
    females_index = np.array(females_stat.index)

    males_data = corr_harmo[males_index]
    females_data = corr_harmo[females_index]

    print(f'Male harmonized matrix: {males_data.shape}, Female harmonized matrix: {females_data.shape}')

    #rois_dim = np.max(a_dims[:, 1])
    dx_data_males = y_target[males_index]
    dx_data_females = y_target[females_index]

    # Equal series length

    x_data_males = equal_len(males_data)
    x_data_females = equal_len(females_data)

    print(f'Final X_data shape from males: {x_data_males.shape}, Final X_data shape from females: {x_data_females.shape}')

    return x_data_males, dx_data_females, x_data_females, dx_data_females



    
